In [1]:
import datetime as dt
import numpy as np
import pandas as pd

In [2]:
#from BasicFin import DayCount, disctorates, ratestodics, refactordates
from SmithWilson import SWExtrapolate
from ImportData import importSWEiopa
from BondClasses import ZeroCouponBond,BondPrices

In [2]:
paramfile = pd.read_csv("Parameters.csv")
paramfile.index = paramfile["Parameter_name"]
del paramfile["Parameter_name"]

# IMPORT EIOPA CURVE

In [3]:
selected_param_file = paramfile.loc["EIOPA_param_file"][0]
selected_curves_file = paramfile.loc["EIOPA_curves_file"][0]
country = paramfile.loc["country"][0]
compounding = paramfile["compounding"]

[maturities_country, curve_country, extra_param, Qb]= importSWEiopa(selected_param_file, selected_curves_file, country)

In [4]:
# Maturity of observations:
M_Obs = np.transpose(np.array(maturities_country.values))

# Ultimate froward rate ufr represents the rate to which the rate curve will converge as time increases:
ufr = extra_param.iloc[3]/100

# Convergence speed parameter alpha controls the speed at which the curve converges towards the ufr from the last liquid point:
alpha = extra_param.iloc[4]

# Qb calibration vector published by EIOPA for the curve calibration:
Qb = np.transpose(np.array(Qb.values))

# PRICING OF A COUPON BOND

In [5]:
# Model parameters
MD = dt.date(2023,4,29)

# Example CouponBond
issuedate = dt.date(2022,12,3)
maturitydate = dt.date(2025,12,3)
notional = 100
couponrate = 0.05
frequency = 1
recovrate = 0.7
defprob =0.05
zspread = 0.005
zcb = ZeroCouponBond(issuedate, maturitydate, frequency, notional, couponrate, recovrate, defprob, zspread)

In [6]:
zcb.createcashflows()
zcbPrice = BondPrices(MD,zspread,compounding)

In [7]:
[datesconsidered,datefrac] = zcbPrice.refactordates(zcb.coupondates,MD)
[datesconsiderednot,datefracnot] = zcbPrice.refactordates(zcb.notionaldates,MD)

[datetime.timedelta(days=-147) datetime.timedelta(days=218)
 datetime.timedelta(days=584) datetime.timedelta(days=949)]
[datetime.timedelta(days=949)]


In [8]:
targetratescoupon = SWExtrapolate(np.transpose(datefrac),M_Obs, Qb, ufr, alpha)

In [9]:
targetratesnotional = SWExtrapolate(np.transpose(datefracnot),M_Obs, Qb, ufr, alpha)

In [10]:
zcbPrice.PriceBond(targetratescoupon,targetratesnotional,datefrac,datefracnot,zcb.couponcf[datesconsidered],zcb.notionalcf)

In [11]:
zcbPrice.marketprice

array([106.237753])